# Advanced SVIPro: Strategy Comparison and Optimization

**Advanced techniques for spatial sampling design and optimization**

This notebook covers:

1. Comparing multiple sampling strategies
2. Optimizing spacing for target sample size
3. Using performance tools for large datasets
4. Error handling and edge cases
5. Integration with external tools

---

In [ ]:
# Import all necessary libraries
import svipro
from svipro import (
    GridSampling, RoadNetworkSampling, SamplingConfig,
    compare_strategies, plot_coverage_statistics,
    SVIProError, BoundaryError, ConfigurationError
)

import geopandas as gpd
from shapely.geometry import box
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print(f"SVIPro version: {svipro.__version__}")

## 1. Strategy Comparison Framework

Compare multiple sampling strategies on the same study area.

In [ ]:
# Define study area (larger area for comparison)
study_area = box(9.1, 45.4, 9.3, 45.6)  # ~20km x ~20km area

print(f"Study area bounds: {study_area.bounds}")
print(f"Study area: {study_area.area:.4f} square degrees")

In [ ]:
# Create multiple strategies with different parameters
strategies = {
    'Grid (50m)': GridSampling(SamplingConfig(spacing=50, seed=42)),
    'Grid (100m)': GridSampling(SamplingConfig(spacing=100, seed=42)),
    'Grid (200m)': GridSampling(SamplingConfig(spacing=200, seed=42)),
}

print(f"Created {len(strategies)} strategies for comparison")

In [ ]:
# Generate samples for each strategy
results = {}

for name, strategy in strategies.items():
    print(f"Generating {name}...")
    points = strategy.generate(study_area)
    results[name] = {
        'strategy': strategy,
        'points': points,
        'n_points': len(points),
        'metrics': strategy.calculate_coverage_metrics()
    }

print(f"\nGenerated samples for all {len(results)} strategies")

### 1.1 Visual Comparison

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, (name, data) in enumerate(results.items()):
    ax = axes[idx]
    
    # Plot boundary
    gpd.GeoSeries([study_area]).plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
    
    # Plot points
    data['points'].plot(ax=ax, markersize=5, alpha=0.6)
    
    ax.set_title(f"{name}\n{data['n_points']} points")
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

plt.tight_layout()
plt.show()

### 1.2 Metrics Comparison Table

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    name: {
        'Points': data['n_points'],
        'Density (pts/km²)': data['metrics']['density_pts_per_km2'],
        'Area (km²)': data['metrics']['area_km2'],
    }
    for name, data in results.items()
}).T

print("Strategy Comparison:")
print(comparison_df.round(2))

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Points comparison
comparison_df['Points'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Number of Sample Points')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Density comparison
comparison_df['Density (pts/km²)'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_title('Sampling Density')
axes[1].set_ylabel('pts/km²')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 2. Optimizing Spacing for Target Sample Size

Use the `optimize_spacing_for_target_n()` method to achieve a desired number of samples.

In [ ]:
# Define target number of points
target_n = 500

print(f"Target: approximately {target_n} sample points")
print("Optimizing spacing...")

# Create strategy with initial guess
strategy = GridSampling(SamplingConfig(spacing=100, seed=42))

# Optimize spacing
optimized_points = strategy.optimize_spacing_for_target_n(
    study_area,
    target_n=target_n,
    min_spacing=20,
    max_spacing=500
)

print(f"\nOptimized spacing: {strategy.config.spacing:.2f} meters")
print(f"Actual points generated: {len(optimized_points)}")
print(f"Target: {target_n} points")
print(f"Difference: {abs(len(optimized_points) - target_n)} points ({abs(len(optimized_points) - target_n)/target_n*100:.1f}%)")

### 2.1 Visualize Optimized Result

In [ ]:
# Plot optimized sampling
fig, ax = plt.subplots(figsize=(10, 10))

gpd.GeoSeries([study_area]).plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
optimized_points.plot(ax=ax, markersize=10, color='green', alpha=0.6)

ax.set_title(f'Optimized Grid Sampling\n{len(optimized_points)} points (~{strategy.config.spacing:.0f}m spacing)', fontsize=14)
plt.show()

## 3. Error Handling and Edge Cases

Learn to handle common errors and edge cases.

In [ ]:
# Example 1: Handling invalid configuration
try:
    invalid_config = SamplingConfig(spacing=-100)
except ConfigurationError as e:
    print(f"Caught error: {e}")
    print(f"Details: {e.details}")

In [ ]:
# Example 2: Handling small boundary
from svipro import handle_small_boundary

# Very small boundary
tiny_boundary = box(0, 0, 0.001, 0.001)

try:
    # Try to handle small boundary
    processed, modified = handle_small_boundary(tiny_boundary, spacing=100)
    print(f"Boundary was modified: {modified}")
except BoundaryError as e:
    print(f"Boundary too small: {e}")

In [ ]:
# Example 3: Validating spacing bounds
from svipro import check_spacing_bounds

# Test various spacing values
test_spacings = [0.5, 50, 100, 500, 20000]

for spacing in test_spacings:
    try:
        check_spacing_bounds(spacing)
        print(f"✓ {spacing}m: Valid")
    except ConfigurationError as e:
        print(f"✗ {spacing}m: {e}")

## 4. Performance Considerations

Using SVIPro's performance tools for large datasets.

In [ ]:
# Estimate processing time
from svipro import estimate_processing_time

# Estimate time for different strategies and point counts
point_counts = [100, 1000, 10000, 50000]

for n in point_counts:
    grid_time = estimate_processing_time(n, 'grid')
    road_time = estimate_processing_time(n, 'road_network')
    print(f"{n:5d} points: Grid={grid_time:.2f}s, Road={road_time:.2f}s")

In [ ]:
# Warn about large output
from svipro import warn_large_output
import warnings

# Test warning system
print("Testing large output warnings:")

with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    
    # Small output - no warning
    warn_large_output(1000)
    print(f"  1000 points: {len(w)} warnings")
    
    # Large output - warning
    warn_large_output(50000)
    print(f"  50000 points: {len(w)} warnings")
    if w:
        print(f"    Message: {w[-1].message}")

## 5. Integration with External Tools

### 5.1 Loading External Boundaries

In [ ]:
# Load boundary from GeoJSON file (if you have one)
# For this example, we'll create a sample GeoDataFrame
sample_gdf = gpd.GeoDataFrame(
    {'name': ['Study Area'], 'geometry': [study_area]},
    crs='EPSG:4326'
)

# Save to file
sample_gdf.to_file('study_area.geojson', driver='GeoJSON')
print("Created sample study_area.geojson")

# Load back
loaded_gdf = gpd.read_file('study_area.geojson')
print(f"Loaded {len(loaded_gdf)} features")

# Extract boundary
boundary_from_file = loaded_gdf.geometry.iloc[0]
print(f"Boundary type: {type(boundary_from_file)}")

### 5.2 Export to Different Formats

In [ ]:
# Generate samples
strategy = GridSampling(SamplingConfig(spacing=200, seed=42))
points = strategy.generate(study_area)

# Export to different formats

# 1. GeoJSON
points.to_file('samples.geojson', driver='GeoJSON')
print("✓ Exported to samples.geojson")

# 2. CSV (with coordinates)
points_csv = points.copy()
points_csv['lon'] = points.geometry.x
points_csv['lat'] = points.geometry.y
points_csv.drop('geometry', axis=1).to_csv('samples.csv', index=False)
print("✓ Exported to samples.csv")

# 3. Shapefile
# points.to_file('samples.shp', driver='ESRI Shapefile')
# print("✓ Exported to samples.shp")

### 5.3 Integration with Pandas

In [ ]:
# Convert to pandas DataFrame for analysis
df = pd.DataFrame(points.drop('geometry', axis=1))

print("Sample data as DataFrame:")
print(df.head(10))

# Basic statistics
print(f"\nDataFrame shape: {df.shape}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nSummary statistics:\n{df.describe()}")

## 6. Reproducibility Workflow

Create a complete, reproducible sampling workflow.

In [ ]:
# Define reproducible workflow
def reproducible_sampling_workflow(boundary, spacing, seed, output_prefix):
    """
    Complete reproducible sampling workflow.
    
    Args:
        boundary: Study area boundary
        spacing: Sampling spacing in meters
        seed: Random seed for reproducibility
        output_prefix: Prefix for output files
    
    Returns:
        Dictionary with results and metadata
    """
    # Configuration
    config = SamplingConfig(spacing=spacing, seed=seed)
    
    # Generate samples
    strategy = GridSampling(config)
    points = strategy.generate(boundary)
    
    # Calculate metrics
    metrics = strategy.calculate_coverage_metrics()
    
    # Export
    geojson_path = f"{output_prefix}_samples.geojson"
    strategy.to_geojson(geojson_path, include_metadata=True)
    
    # Return results
    return {
        'config': config.to_dict(),
        'points': points,
        'metrics': metrics,
        'output_file': geojson_path,
        'strategy': strategy.__class__.__name__
    }

# Run workflow
results = reproducible_sampling_workflow(
    boundary=study_area,
    spacing=150,
    seed=12345,
    output_prefix='milan_reproducible'
)

print("Reproducible workflow completed:")
print(f"  Strategy: {results['strategy']}")
print(f"  Points: {results['metrics']['n_points']}")
print(f"  Output: {results['output_file']}")
print(f"  Seed: {results['config']['seed']}")

## 7. Summary and Best Practices

### Key Takeaways

1. **Always use seeds** for reproducibility
2. **Validate inputs** before processing
3. **Handle errors** gracefully
4. **Document everything** - config, parameters, results
5. **Test on small areas** before scaling up

### Performance Tips

- Use appropriate CRS for accurate spacing
- Consider point count vs. accuracy trade-offs
- Use performance tools for large datasets
- Cache results when possible

### Common Pitfalls

- ❌ Not checking boundary validity
- ❌ Using inappropriate CRS
- ❌ Not documenting configuration
- ❌ Ignoring error messages
- ❌ Not testing on small data first

---

**Next Steps:**
- Try with your own study area
- Integrate into your research pipeline
- Explore the API documentation
- Check out the case studies

**Happy Sampling! 🎉**